## 0. Setup
Import the Python modules used throughout the notebook. Make sure you have already installed the packages listed in the README (pandas, numpy, mutagen, unidecode).

### Package bootstrap
Install any missing Python packages required by this workflow so the import cell succeeds even on a fresh environment.

In [ ]:
import importlib
import subprocess
import sys

REQUIRED_PACKAGES = {
    "pandas": "pandas",
    "numpy": "numpy",
    "mutagen": "mutagen",
    "unidecode": "Unidecode"
}

for module_name, install_name in REQUIRED_PACKAGES.items():
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing missing dependency: {install_name}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", install_name])
print("Dependency check complete.")

In [ ]:
from __future__ import annotations
from collections import defaultdict
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Iterable, List, Optional

import math
import re
import shutil
import pandas as pd
import numpy as np
from mutagen import File as MutagenFile
from unidecode import unidecode

## 1. Configuration
Define key directories (relative to the repository root) and ensure the output folder for reports exists.

In [ ]:
# Adjust these paths if you relocate folders.
REPO_ROOT = Path.cwd()
DOWNLOAD_ROOT = REPO_ROOT / "Downloaded"
SPOTIFY_PLAYLISTS = REPO_ROOT / "spotify_playlists"
SHOPPING_LIST_DIR = REPO_ROOT / "shopping_lists"
LIBRARY_INDEX_CSV = REPO_ROOT / "library_index.csv"
ADD_ROOT = REPO_ROOT / "Add"

SHOPPING_LIST_DIR.mkdir(exist_ok=True)
ADD_ROOT.mkdir(exist_ok=True)
print(f"Repository root: {REPO_ROOT}")
print(f"Download library: {DOWNLOAD_ROOT}")
print(f"Spotify playlist CSVs: {SPOTIFY_PLAYLISTS}")
print(f"Shopping/output directory: {SHOPPING_LIST_DIR}")
print(f"Add drop directory: {ADD_ROOT}")

## 2. Helper functions
Canonicalization helpers keep matching consistent between Spotify exports and local audio metadata.

In [ ]:
NON_ALNUM = re.compile(r"[^a-z0-9]+")
FEAT_PATTERN = re.compile(r"\(feat\..*?\)", re.IGNORECASE)
REMIX_PATTERN = re.compile(r"-\s*(remaster(ed)?|remix|edit|mix).*", re.IGNORECASE)
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.m4a', '.aac', '.ogg', '.wav', '.aiff'}
INVALID_PATH_CHARS = re.compile(r"[<>:\"/\\|?*]")


def canonicalize_string(value: Optional[str]) -> str:
    if not value:
        return ""
    normalized = unidecode(str(value))
    normalized = FEAT_PATTERN.sub("", normalized)
    normalized = REMIX_PATTERN.sub("", normalized)
    normalized = normalized.lower()
    normalized = NON_ALNUM.sub(" ", normalized)
    normalized = normalized.strip()
    return re.sub(r"\s+", " ", normalized)


def safe_path_component(value: Optional[str], fallback: str = "Unknown") -> str:
    candidate = str(value).strip() if value else fallback
    candidate = unidecode(candidate)
    candidate = INVALID_PATH_CHARS.sub("", candidate)
    candidate = candidate.replace('/', '-').replace('\\', '-')
    candidate = candidate.strip()
    return candidate or fallback


def primary_artist(artists_field: Optional[str]) -> str:
    if not artists_field or not isinstance(artists_field, str):
        return ""
    first = artists_field.split(';')[0]
    return first.strip()


def friendly_playlist_name(csv_path: Path) -> str:
    name = csv_path.stem.replace('_', ' ')
    return name.strip()


def duration_ms_from_audio(audio_obj) -> Optional[int]:
    if audio_obj and audio_obj.info and getattr(audio_obj.info, 'length', None):
        return int(round(audio_obj.info.length * 1000))
    return None

## 3. Process new additions
Move fresh MP3 downloads from the staging `Add/` folder into the canonical `Downloaded/Artist/Album/Title.mp3` structure before scanning the library.

In [ ]:
SUPPORTED_IMPORT_SUFFIXES = {'.mp3'}


def tag_value(tag) -> Optional[str]:
    if tag is None:
        return None
    if hasattr(tag, 'text') and getattr(tag, 'text'):
        return str(tag.text[0])
    if isinstance(tag, (list, tuple)) and tag:
        return str(tag[0])
    if isinstance(tag, str):
        return tag
    return None


def process_new_additions(add_root: Path, download_root: Path) -> pd.DataFrame:
    if not add_root.exists():
        print(f"Add directory not found: {add_root}")
        return pd.DataFrame()

    mp3_files = sorted(add_root.rglob('*.mp3'))
    if not mp3_files:
        print(f"No new MP3 files found under {add_root}.")
        return pd.DataFrame()

    actions = []
    for src_path in mp3_files:
        rel_src = src_path.relative_to(add_root)
        if not src_path.exists():
            message = "Source file missing before processing"
            print(f"Skipping {rel_src}: {message}")
            actions.append({
                'source': rel_src.as_posix(),
                'destination': None,
                'status': 'skipped_missing_source',
                'message': message
            })
            continue
        try:
            audio = MutagenFile(src_path)
        except Exception as exc:
            message = f"Unable to read metadata: {exc}"
            print(f"Skipping {rel_src}: {message}")
            actions.append({
                'source': rel_src.as_posix(),
                'destination': None,
                'status': 'error_read',
                'message': message
            })
            continue

        tags = getattr(audio, 'tags', None) if audio else None
        artist_raw = tag_value(tags.get('TPE1')) if tags else None
        artist_raw = artist_raw or tag_value(tags.get('artist')) if tags else artist_raw
        album_raw = tag_value(tags.get('TALB')) if tags else None
        album_raw = album_raw or tag_value(tags.get('album')) if tags else album_raw
        title_raw = tag_value(tags.get('TIT2')) if tags else None
        title_raw = title_raw or tag_value(tags.get('title')) if tags else title_raw

        artist_component = safe_path_component(artist_raw, 'Unknown Artist')
        album_component = safe_path_component(album_raw, 'Unknown Album')
        title_value = title_raw or src_path.stem
        title_component = safe_path_component(title_value, src_path.stem)
        title_canonical = canonicalize_string(title_value)

        dest_dir = download_root / artist_component / album_component
        dest_dir.mkdir(parents=True, exist_ok=True)

        duplicate_path = None
        for existing_path in dest_dir.glob('*'):
            if not existing_path.is_file() or existing_path.suffix.lower() not in AUDIO_EXTENSIONS:
                continue
            try:
                existing_audio = MutagenFile(existing_path)
            except Exception:
                existing_audio = None
            existing_tags = getattr(existing_audio, 'tags', None) if existing_audio else None
            existing_title = None
            if existing_tags:
                existing_title = tag_value(existing_tags.get('TIT2'))
                if not existing_title:
                    existing_title = tag_value(existing_tags.get('title'))
            if not existing_title:
                existing_title = existing_path.stem
            existing_canonical = canonicalize_string(existing_title)
            if existing_canonical == title_canonical:
                duplicate_path = existing_path
                break

        if duplicate_path is not None:
            message = f"Title already exists: {duplicate_path.relative_to(download_root)}"
            print(f"Skipping {rel_src}: {message}")
            actions.append({
                'source': rel_src.as_posix(),
                'destination': duplicate_path.relative_to(download_root).as_posix(),
                'status': 'skipped_duplicate_title',
                'message': message
            })
            continue

        dest_path = dest_dir / f"{title_component}{src_path.suffix.lower()}"

        if dest_path.exists():
            message = f"Destination already exists: {dest_path}"
            print(f"Skipping {rel_src}: {message}")
            actions.append({
                'source': rel_src.as_posix(),
                'destination': dest_path.relative_to(download_root).as_posix(),
                'status': 'skipped_exists',
                'message': message
            })
            continue

        try:
            shutil.move(str(src_path), dest_path)
        except FileNotFoundError as exc:
            message = f"Move failed (missing path): {exc}"
            print(f"Skipping {rel_src}: {message}")
            actions.append({
                'source': rel_src.as_posix(),
                'destination': dest_path.relative_to(download_root).as_posix(),
                'status': 'error_move',
                'message': message
            })
            continue
        except OSError as exc:
            message = f"Move failed: {exc}"
            print(f"Skipping {rel_src}: {message}")
            actions.append({
                'source': rel_src.as_posix(),
                'destination': dest_path.relative_to(download_root).as_posix(),
                'status': 'error_move',
                'message': message
            })
            continue

        actions.append({
            'source': rel_src.as_posix(),
            'destination': dest_path.relative_to(download_root).as_posix(),
            'status': 'moved',
            'message': 'Moved into library'
        })

    # Clean up any empty directories left behind in Add/
    for folder in sorted(add_root.rglob('*'), reverse=True):
        if folder.is_dir():
            try:
                next(folder.iterdir())
            except StopIteration:
                folder.rmdir()

    result_df = pd.DataFrame(actions)
    moved_count = (result_df['status'] == 'moved').sum() if not result_df.empty else 0
    skipped_exists_count = (result_df['status'] == 'skipped_exists').sum() if not result_df.empty else 0
    duplicate_count = (result_df['status'] == 'skipped_duplicate_title').sum() if not result_df.empty else 0
    missing_source_count = (result_df['status'] == 'skipped_missing_source').sum() if not result_df.empty else 0
    error_read_count = (result_df['status'] == 'error_read').sum() if not result_df.empty else 0
    error_move_count = (result_df['status'] == 'error_move').sum() if not result_df.empty else 0
    print(
        f"Processed {len(actions)} files | Moved: {moved_count} | "
        f"Skipped (already existed): {skipped_exists_count} | "
        f"Skipped (duplicate title): {duplicate_count} | "
        f"Skipped (missing source): {missing_source_count} | "
        f"Errors (metadata read): {error_read_count} | Errors (move): {error_move_count}"
    )
    return result_df


new_additions_log = process_new_additions(ADD_ROOT, DOWNLOAD_ROOT)
if not new_additions_log.empty:
    display(new_additions_log)

## 4. Scan the downloaded library
Create or refresh an auditable `library_index.csv` capturing metadata for every audio file under `Downloaded/`.

In [ ]:
def scan_downloaded_library(download_root: Path) -> pd.DataFrame:
    records = []
    if not download_root.exists():
        print(f"Download directory not found: {download_root}")
        return pd.DataFrame()

    for file_path in download_root.rglob('*'):
        if not file_path.is_file() or file_path.suffix.lower() not in AUDIO_EXTENSIONS:
            continue
        try:
            audio = MutagenFile(file_path)
        except Exception as exc:
            print(f"Failed to read {file_path}: {exc}")
            audio = None

        tags = getattr(audio, 'tags', None) if audio else None
        artist_tag = None
        title_tag = None
        album_tag = None

        if tags:
            artist_tag = tags.get('TPE1') or tags.get('artist')
            title_tag = tags.get('TIT2') or tags.get('title')
            album_tag = tags.get('TALB') or tags.get('album')

        artist_str = str(artist_tag.text[0]) if hasattr(artist_tag, 'text') else (artist_tag if isinstance(artist_tag, str) else None)
        title_str = str(title_tag.text[0]) if hasattr(title_tag, 'text') else (title_tag if isinstance(title_tag, str) else None)
        album_str = str(album_tag.text[0]) if hasattr(album_tag, 'text') else (album_tag if isinstance(album_tag, str) else None)

        # Fallbacks from the path structure
        if not artist_str:
            artist_str = file_path.parent.name
        if not title_str:
            title_str = file_path.stem

        records.append({
            'file_path': file_path.relative_to(download_root).as_posix(),
            'artist_raw': artist_str,
            'title_raw': title_str,
            'album_raw': album_str,
            'artist_canonical': canonicalize_string(artist_str),
            'title_canonical': canonicalize_string(title_str),
            'duration_ms': duration_ms_from_audio(audio)
        })

    df = pd.DataFrame.from_records(records)
    if not df.empty:
        df.sort_values(['artist_canonical', 'title_canonical', 'file_path'], inplace=True)
    return df

library_index = scan_downloaded_library(DOWNLOAD_ROOT)
print(f"Indexed {len(library_index):,} local tracks")
if not library_index.empty:
    library_index.to_csv(LIBRARY_INDEX_CSV, index=False)
    display(library_index.head())
else:
    print('Library index is empty – check DOWNLOAD_ROOT or file extensions.')

## 5. Load Spotify playlist exports
Combine all CSV files in `spotify_playlists/` into a single DataFrame with helpful flags.

In [ ]:
def load_spotify_playlists(csv_root: Path) -> pd.DataFrame:
    rows = []
    if not csv_root.exists():
        print(f"Spotify playlist directory not found: {csv_root}")
        return pd.DataFrame()

    csv_files = sorted(csv_root.glob('*.csv'))
    if not csv_files:
        print(f"No CSV files found in {csv_root}")
        return pd.DataFrame()

    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
        except Exception as exc:
            print(f"Failed to read {csv_file}: {exc}")
            continue
        df['playlist_name'] = friendly_playlist_name(csv_file)
        df['is_liked'] = csv_file.name.lower() == 'liked_songs.csv'
        df['is_top_songs'] = csv_file.name.lower().startswith('your_top_songs_')
        rows.append(df)

    merged = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()
    if merged.empty:
        return merged

    merged['primary_artist'] = merged['Artist Name(s)'].apply(primary_artist)
    merged['artist_canonical'] = merged['primary_artist'].apply(canonicalize_string)
    merged['title_canonical'] = merged['Track Name'].apply(canonicalize_string)
    return merged

spotify_df = load_spotify_playlists(SPOTIFY_PLAYLISTS)
print(f"Loaded {len(spotify_df):,} Spotify rows across {spotify_df['playlist_name'].nunique() if not spotify_df.empty else 0} playlists")
if not spotify_df.empty:
    display(spotify_df.head())

## 6. Match Spotify tracks to the local library
Left-join on canonical artist/title keys and filter by duration tolerance where available.

In [ ]:
DURATION_TOLERANCE_MS = 3000

def match_tracks(spotify_df: pd.DataFrame, library_df: pd.DataFrame, duration_tolerance_ms: int = DURATION_TOLERANCE_MS) -> pd.DataFrame:
    if spotify_df.empty:
        return pd.DataFrame()
    if library_df.empty:
        result = spotify_df.copy()
        result['file_path'] = pd.NA
        result['duration_ms_local'] = pd.NA
        return result

    lib_cols = library_df.rename(columns={'duration_ms': 'duration_ms_local'})
    merged = spotify_df.merge(lib_cols, how='left', on=['artist_canonical', 'title_canonical'], suffixes=('_spotify', '_local'))

    if 'duration_ms_local' in merged.columns:
        mask = merged['duration_ms_local'].notna() & merged['Duration (ms)'].notna()
        mismatched = mask & (merged['Duration (ms)'] - merged['duration_ms_local']).abs() > duration_tolerance_ms
        merged.loc[mismatched, ['file_path', 'duration_ms_local']] = pd.NA
    return merged

matched_df = match_tracks(spotify_df, library_index)
print(f"Matched rows: {len(matched_df):,}")
if not matched_df.empty:
    have_files = matched_df['file_path'].notna().sum()
    print(f"Tracks already downloaded: {have_files:,}")
    print(f"Tracks missing locally: {len(matched_df) - have_files:,}")
    display(matched_df.head())

## 7. Generate a dated shopping list
Aggregate missing tracks across playlists and export a timestamped CSV in `shopping_lists/`.

In [ ]:
def build_shopping_list(matched_df: pd.DataFrame) -> pd.DataFrame:
    if matched_df.empty:
        return pd.DataFrame()
    missing = matched_df[matched_df['file_path'].isna()].copy()
    if missing.empty:
        return pd.DataFrame()

    grouped = (
        missing.groupby(['artist_canonical', 'title_canonical'], as_index=False)
        .agg({
            'primary_artist': 'first',
            'Track Name': 'first',
            'Album Name': lambda col: col.dropna().iloc[0] if col.dropna().any() else pd.NA,
            'Duration (ms)': 'first',
            'playlist_name': lambda col: sorted(set(col)),
            'is_liked': 'any',
            'is_top_songs': 'any'
        })
    )
    grouped['Playlists_Count'] = grouped['playlist_name'].apply(len)
    grouped['Playlists'] = grouped['playlist_name'].apply(lambda names: '; '.join(names))
    grouped.rename(columns={
        'primary_artist': 'Artist',
        'Track Name': 'Title',
        'Album Name': 'Album',
        'Duration (ms)': 'Duration_ms',
        'is_liked': 'Is_Liked',
        'is_top_songs': 'Is_Top_Songs'
    }, inplace=True)
    columns = ['Artist', 'Title', 'Album', 'Duration_ms', 'Playlists_Count', 'Playlists', 'Is_Liked', 'Is_Top_Songs']
    grouped = grouped[columns]
    grouped.sort_values(['Playlists_Count', 'Is_Liked', 'Artist', 'Title'], ascending=[False, False, True, True], inplace=True)
    return grouped

shopping_df = build_shopping_list(matched_df)
if shopping_df.empty:
    print('All playlist tracks already exist locally – no shopping list generated.')
else:
    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    shopping_path = SHOPPING_LIST_DIR / f'shopping_list_{timestamp}.csv'
    shopping_df.to_csv(shopping_path, index=False)
    print(f"Shopping list saved to {shopping_path}")
    display(shopping_df.head())

## 8. Generate an orphaned-tracks list
Highlight tracks that exist in `Downloaded/` but are not referenced by any current playlist snapshot.

In [ ]:
def build_orphaned_tracks(matched_df: pd.DataFrame, library_df: pd.DataFrame) -> pd.DataFrame:
    if library_df.empty:
        return pd.DataFrame()
    playlist_keys = set(zip(matched_df['artist_canonical'], matched_df['title_canonical'])) if not matched_df.empty else set()
    library_df = library_df.copy()
    library_df['key'] = list(zip(library_df['artist_canonical'], library_df['title_canonical']))
    mask = library_df['key'].apply(lambda key: key not in playlist_keys)
    orphaned = library_df[mask].copy()
    if orphaned.empty:
        return pd.DataFrame()
    orphaned.rename(columns={
        'artist_raw': 'Artist',
        'title_raw': 'Title',
        'album_raw': 'Album',
        'duration_ms': 'Duration_ms'
    }, inplace=True)
    columns = ['Artist', 'Title', 'Album', 'Duration_ms', 'file_path']
    return orphaned[columns]

orphan_df = build_orphaned_tracks(matched_df, library_index)
if orphan_df.empty:
    print('No orphaned tracks – every local track appears in at least one playlist snapshot.')
else:
    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    orphan_path = SHOPPING_LIST_DIR / f'orphaned_tracks_{timestamp}.csv'
    orphan_df.to_csv(orphan_path, index=False)
    print(f"Orphaned-track report saved to {orphan_path}")
    display(orphan_df.head())

## 9. Show Playlist Statistics
Summarize key statistics about each playlist, including total tracks, matched tracks, missing tracks, and orphaned tracks.

In [ ]:
if 'matched_df' not in globals():
    print("Run cells 1-7 to create 'matched_df' before generating playlist stats.")
elif matched_df.empty:
    print('Playlist DataFrame is empty – load Spotify CSVs first.')
else:
    stats = (
        matched_df
        .groupby('playlist_name', dropna=False)
        .agg(
            Total_Tracks=('Track Name', 'size'),
            Matched_Tracks=('file_path', lambda col: col.notna().sum()),
            Liked_Snapshot=('is_liked', 'any'),
            Top_Songs_Snapshot=('is_top_songs', 'any')
        )
        .reset_index()
    )
    stats['Missing_Tracks'] = stats['Total_Tracks'] - stats['Matched_Tracks']
    stats['Percent_Complete'] = (stats['Matched_Tracks'] / stats['Total_Tracks'] * 100).round(1)
    stats.sort_values(['Percent_Complete', 'playlist_name'], ascending=[False, True], inplace=True)

    overall_total = int(stats['Total_Tracks'].sum())
    overall_missing = int(stats['Missing_Tracks'].sum())
    overall_matched = overall_total - overall_missing

    missing_unique = (
        matched_df[matched_df['file_path'].isna()]
        .drop_duplicates(subset=['artist_canonical', 'title_canonical'])
        .shape[0]
    )

    print(
        f"Playlists analyzed: {len(stats)} | Tracks: {overall_total:,} | "
        f"Matched: {overall_matched:,} | Missing: {overall_missing:,}"
    )
    print(f"Unique missing tracks across all playlists: {missing_unique:,}")
    display(stats)


## 10. Generate Playlists
Build on these DataFrames to generate Innioasis Y1 playlist files (`.m3u8`) containing all the real tracks.

In [ ]:
PLAYLIST_EXPORT_DIR = REPO_ROOT / "generated_playlists"
PLAYLIST_RELATIVE_ROOT = Path("Music")  # Adjust if your device expects a different root folder.
PLAYLIST_EXPORT_DIR.mkdir(exist_ok=True)

if 'matched_df' not in globals():
    print("Run cells 1-7 to build 'matched_df' before exporting playlists.")
elif matched_df.empty:
    print('matched_df is empty – load Spotify CSVs and re-run matching first.')
else:
    def playlist_filename(name: str) -> str:
        safe = re.sub(r"[^A-Za-z0-9._-]+", "_", (name or "playlist").strip())
        safe = safe.strip('_') or "playlist"
        return f"{safe}.m3u8"

    exports = []
    grouped = matched_df.groupby('playlist_name', sort=False)
    for playlist_name, group in grouped:
        resolved = group[group['file_path'].notna()].copy()
        if resolved.empty:
            continue
        if 'Position' in resolved.columns:
            resolved.sort_values('Position', inplace=True)
        else:
            resolved = resolved.sort_index()

        playlist_path = PLAYLIST_EXPORT_DIR / playlist_filename(playlist_name)
        lines = ['#EXTM3U']
        for _, row in resolved.iterrows():
            duration_sec = int(round(row['Duration (ms)'] / 1000)) if pd.notna(row.get('Duration (ms)')) else -1
            artist = row.get('primary_artist') or row.get('Artist Name(s)') or 'Unknown Artist'
            title = row.get('Track Name') or 'Unknown Title'
            lines.append(f"#EXTINF:{duration_sec},{artist} - {title}")
            device_path = (PLAYLIST_RELATIVE_ROOT / Path(row['file_path'])).as_posix()
            lines.append(device_path)

        playlist_path.write_text('\n'.join(lines) + '\n', encoding='utf-8')
        exports.append({
            'playlist_name': playlist_name,
            'tracks_total': int(len(group)),
            'tracks_written': int(len(resolved)),
            'tracks_missing': int(group['file_path'].isna().sum()),
            'output_file': playlist_path.relative_to(REPO_ROOT).as_posix()
        })

    if not exports:
        print('No playlists had matched tracks – nothing exported.')
    else:
        summary = pd.DataFrame(exports)
        summary.sort_values('playlist_name', inplace=True)
        print(f"Exported {len(summary)} playlists to {PLAYLIST_EXPORT_DIR}")
        display(summary)
